In [1]:
import sys
from pathlib import Path
import os

In [2]:
file = Path(os.path.abspath('')).resolve()
parent, root = file.parent, file.parents[0]
sys.path.append(str(root))

In [72]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from libs.utils import get_rings
from PIL import Image, ImageDraw, ImageFont, ImageOps
from skimage import exposure
import numpy as np

In [4]:
biomarker = 'mm6_srf'
threshold = 0.991
N_SLICES = 49
N_PIXEL_H = 496
N_PIXEL_W = 512
ring_diameters = [1, 3, 6]
clahe = True
folder_in_results = '111853 clahe'
filename = 'validation_results_20210212-111853.csv'
ring_extract = biomarker.split('_')[0]

In [5]:
file_path = f'../results/{folder_in_results}/{filename}'

In [6]:
data = pd.read_csv(file_path)
data = data.rename(columns={'Unnamed: 0': 'image_name', 'Unnamed: 1': 'bscan'})

In [7]:
datasets_path = file.parents[2].joinpath('Datasets', 'Location')

In [8]:
mask_generator = get_rings(ring_diameters, N_SLICES, N_PIXEL_H, N_PIXEL_W)

In [9]:
y_true = data[biomarker + '_true'].to_list()
y_pred = data[biomarker + '_pred'].to_list()

In [10]:
data['FN'] = [1 if y_pred[i] < threshold and y_true[i] == 1 else 0 for i in range(len(y_true))]
data['FP'] = [1 if y_pred[i] > threshold and y_true[i] == 0 else 0 for i in range(len(y_true))]
data['TP'] = [1 if y_pred[i] > threshold and y_true[i] == 1 else 0 for i in range(len(y_true))]

In [11]:
data.head()

,image_name,bscan,healthy_true,present_srf_true,present_irf_true,mm1_srf_true,mm3_srf_true,mm6_srf_true,mm1_irf_true,mm3_irf_true,...,present_irf_pred,mm1_srf_pred,mm3_srf_pred,mm6_srf_pred,mm1_irf_pred,mm3_irf_pred,mm6_irf_pred,FN,FP,TP
0,2150-001-OD-FE-SCR-KESTREL-191623.f540fea,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001248,0.0,0.0,0.000449,0.0,0.0,0.005327,0,0,0
1,2150-001-OD-FE-SCR-KESTREL-191623.f540fea,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000440,0.0,0.0,0.000420,0.0,0.0,0.002366,0,0,0
2,2150-001-OD-FE-SCR-KESTREL-191623.f540fea,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000218,0.0,0.0,0.001383,0.0,0.0,0.001641,0,0,0
3,2150-001-OD-FE-SCR-KESTREL-191623.f540fea,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000685,0.0,0.0,0.000795,0.0,0.0,0.003827,0,0,0
4,2150-001-OD-FE-SCR-KESTREL-191623.f540fea,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000673,0.0,0.0,0.000818,0.0,0.0,0.008639,0,0,0


In [12]:
data_fp = data[data['FP'] == 1].copy().reset_index(drop=True)
data_fn = data[data['FN'] == 1].copy().reset_index(drop=True)
data_tp = data[data['TP'] == 1].copy().reset_index(drop=True)

In [13]:
print(len(data_fn), len(data_tp), len(data_fp))

66 44 50


### Draw lines and save images 

In [14]:
def get_lines(mask):
    lines = {'1mm': [], '3mm': [], '6mm': []}
    for i_ring, ring in enumerate(lines.keys()):
        for i in range(1, len(mask[i_ring])):
            if mask[i_ring][i] != mask[i_ring][i - 1]:
                lines[ring].append(i)
    return lines

In [68]:
def draw_lines(im, lines, type_line):
    draw = ImageDraw.Draw(im)
    text_written = []
    
    if type_line == 'ring':
        width = 3
        fill = 256
        
        for ring in lines.keys():
            for i_line in range(len(lines[ring])):
                line = lines[ring][i_line]
                if line not in text_written:
                    if i_line < len(lines[ring]) // 2:
                        text_written.append(line)
                        draw.text((line + 10, 400), ring, fill=256, font=font)
                    draw.line((line, 35, line, im.size[-1]), fill=256, width=width)
                
    else:
        BLUE = "#0000ff"
        width = 2
        for line in lines:
            draw.line((line, 35, line, im.size[-1]), fill=BLUE, width=width)
                
    return im, draw

In [69]:
font = ImageFont.truetype("Arial.ttf", 22)

In [77]:
def save_modified_files(df, type_df):
    os.makedirs(f'../results/{folder_in_results}/{type_df}_{ring_extract}', exist_ok=True)
    lines_cols = list(range(0, 512, int(512/16)))
    for i_sample in range(len(df)):
        row = df.iloc[i_sample]
        folder_name = row['image_name']
        bscan = row['bscan']
        
        text = 'Present: {:.2f} - 1mm: {:.2f} - 3mm: {:.2f} - 6mm: {:.2f}'.format(
            row['present_srf_pred'], row['mm1_srf_pred'], row['mm3_srf_pred'], row['mm6_srf_pred'])

        image_path = datasets_path.joinpath('extractedTifs', folder_name, 'img_{:02d}.jpg'.format(bscan))
        mask = [mask_generator[i][0, :, bscan] for i in range(len(mask_generator))]

        im = Image.open(image_path)
        if clahe:
            img_adapteq = exposure.equalize_adapthist(np.array(im))
            im = Image.fromarray(np.uint8(img_adapteq*255))

        lines = get_lines(mask)
        im, draw = draw_lines(im, lines, 'ring')
        draw.text((10, 10), text, fill=256, font=font)
        im = im.save(f'../results/{folder_in_results}/{type_df}_{ring_extract}/{folder_name}_{bscan}.jpg')

In [78]:
#save_modified_files(data_tp, 'TP')
save_modified_files(data_fp, 'FP')
#save_modified_files(data_fn, 'FN')